In [1]:
from PIL import Image
import os
from time import time
import multiprocessing
from itertools import repeat
from opticalflow import opticalflow

In [2]:
src = r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\CoarseRegIM\run4_16xr_jpg_rszfc1_padsz1000'
dst = os.path.join(src,'opticalflow_registered_image')
if not os.path.exists(dst): os.mkdir(dst)
ims = [_ for _ in os.listdir(src) if _.endswith('jpg')]
ims2 = [os.path.join(src,_) for _ in os.listdir(src) if _.endswith('jpg')]

In [3]:
inputs1=ims2[::2]
inputs2=ims2[1::2]
inputs1=inputs1[0:4]
inputs2=inputs2[0:4]
inputs1,inputs2

(['\\\\fatherserverdw\\kyuex\\image\\CLUE\\3D study\\he\\CoarseRegIM\\run4_16xr_jpg_rszfc1_padsz1000\\z0001_1C1.jpg',
  '\\\\fatherserverdw\\kyuex\\image\\CLUE\\3D study\\he\\CoarseRegIM\\run4_16xr_jpg_rszfc1_padsz1000\\z0007_7C1.jpg',
  '\\\\fatherserverdw\\kyuex\\image\\CLUE\\3D study\\he\\CoarseRegIM\\run4_16xr_jpg_rszfc1_padsz1000\\z0013_13C1.jpg',
  '\\\\fatherserverdw\\kyuex\\image\\CLUE\\3D study\\he\\CoarseRegIM\\run4_16xr_jpg_rszfc1_padsz1000\\z0019_19C1.jpg'],
 ['\\\\fatherserverdw\\kyuex\\image\\CLUE\\3D study\\he\\CoarseRegIM\\run4_16xr_jpg_rszfc1_padsz1000\\z0004_4C1.jpg',
  '\\\\fatherserverdw\\kyuex\\image\\CLUE\\3D study\\he\\CoarseRegIM\\run4_16xr_jpg_rszfc1_padsz1000\\z0010_10C1.jpg',
  '\\\\fatherserverdw\\kyuex\\image\\CLUE\\3D study\\he\\CoarseRegIM\\run4_16xr_jpg_rszfc1_padsz1000\\z0016_16C1.jpg',
  '\\\\fatherserverdw\\kyuex\\image\\CLUE\\3D study\\he\\CoarseRegIM\\run4_16xr_jpg_rszfc1_padsz1000\\z0022_22C1.jpg'])

In [16]:
start = time()
pool = multiprocessing.Pool(processes=2)
outputs = pool.starmap_async(opticalflow, zip(inputs1, inputs2))
a=[_ for _ in outputs.get()]
print(round(time()-start,2),'sec elapsed') #133sec for registering twice; 270 sec for registering 4times; linear beyond N processes

271.37 sec elapsed


In [4]:
start = time()
pool = multiprocessing.Pool(processes=4)
outputs = pool.starmap_async(opticalflow, zip(inputs1, inputs2))
a=[_ for _ in outputs.get()]
print(round(time()-start,2),'sec elapsed') #

199.17 sec elapsed


In [5]:
start = time()
reg = opticalflow(inputs1[0],inputs2[0])
print(round(time()-start,2),'sec elapsed')

115.13 sec elapsed


In [6]:
start = time()
reg = opticalflow(inputs1[1],inputs2[1])
print(round(time()-start,2),'sec elapsed')

114.63 sec elapsed


In [18]:
ref=np.array(Image.open(os.path.join(src,ims[0])))
mov=np.array(Image.open(os.path.join(src,ims[1])))
refg=rgb2gray(ref)
movg=rgb2gray(mov)
start = time()
v, u = optical_flow_tvl1(refg, movg)
print(round(time()-start,2),'sec elapsed')
nr, nc = refg.shape
row_coords, col_coords = np.meshgrid(np.arange(nr), np.arange(nc),
                                     indexing='ij')
transmap = np.array([row_coords + v, col_coords + u])
# start = time()
# mov_img_warp_ra =[]
# for i in range(3):
#     mov_img_warp = warp(mov[:,:,i],transmap ,mode='edge')
#     mov_img_warp_ra.append(mov_img_warp)
# print(round(time()-start,2),'sec elapsed') #1.5sec

start = time()
pool = multiprocessing.Pool(processes=3)
inputs = [mov[:,:,0],mov[:,:,1],mov[:,:,2]]
outputs = pool.starmap_async(warp, zip(inputs, repeat(transmap)))
print(round(time()-start,2),'sec elapsed') #0.08sec
Image.fromarray(np.stack([(_*255).astype(np.uint8) for _ in outputs.get()],axis=2)).save(os.path.join(dst,ims[1]))

113.76 sec elapsed
0.01 sec elapsed
